In [2]:
import sys
sys.path
sys.path.append(r"c:\users\wannl\appdata\local\packages\pythonsoftwarefoundation.python.3.10_qbz5n2kfra8p0\localcache\local-packages\python310\site-packages")
sys.path.append(r"C:\Users\wannl\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\Scripts")
#c:\users\wannl\appdata\local\packages\pythonsoftwarefoundation.python.3.10_qbz5n2kfra8p0\localcache\local-packages\python310\site-packages
#C:\Users\wannl\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\Scripts

In [ ]:
# Collecting Reddit r/conspiracy dataset
# Code for collection from https://towardsdatascience.com/how-to-collect-a-reddit-dataset-c369de539114

import praw                   # reddit API web scraper app
#from psaw import PushshiftAPI # pushshift wrapper
import time                   # logging progress
import pandas as pd           # creating csv data files
import datetime as dt         # search window

# to use PSAW
#api = PushshiftAPI()

# read-only authentication; to use PRAW
reddit = praw.Reddit(
    client_id="r0UJCKiW44jAd_XBJRXnZQ",
    client_secret="NcIeon56TG06up2kuXS6ZLLkK1wd4g",
    username = "New_Energy7000",
    #password = "",
    user_agent="Web Scraping")

# search window
subreddits = ['conspiracy']
start_year = 2022
end_year = 2022

# directory on which to store data
basecorpus = 'D:\\Research\\my-dataset'

# to check progess
def log_action(action):
    print(action)
    return

## create directory, define search window to input into PSAW
for year in range(start_year, end_year+1):
    action = "[Year]" + str(year)
    log_action(action)

    dirpath = basecorpus + str(year)
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)
    
    # timestamps that define window of posts
    ts_after = int(dt.datetime(year, 1, 1).timestamp())
    ts_before = int(dt.datetime(year+1, 1, 1).timestamp())

## count extraction time for each subreddit, create directory, define path for CSV file
for subreddit in subreddits:
    start_time = time.time()

    action = "\t[Subreddit]" + subreddit
    log_action(action)

    subredditdirpath = dirpath + '\\' + subreddit
    if os.path.exists(subredditdirpath):
        continue
    else:
        os.makedirs(subredditdirpath)

    submissions_csv_path = str(year) + '-' + subreddit + '-submissions.csv'

## define directory where each post will be stored
submissions_dict = {
    "id" : [],
    "url" : [],
    "title" : [],
    "score" : [],
    "num_comments" : [],
    "created_utc" : [],
    "selftext" : [],
    "is_original_content" : [],
    "distinguished" : [],
}

## iterator looping over specific posts to get ids w/ psaw
gen = api.search_submissions(
            after=ts_after,
            before=ts_before,
            filter=['id'],
            subreddit=subreddit,
            limit=None
        )

## use post ids to retrieve full post data from PRAW and append to dictionary
for submission_psaw in gen:
    # psaw
    submission_id = submission_psaw.d_['id']
    # praw
    submission_praw = reddit.submission(id=submission_id)

    submissions_dict["id"].append(submission_praw.id)
    submissions_dict["url"].append(submission_praw.url)
    submissions_dict["title"].append(submission_praw.title)
    submissions_dict["score"].append(submission_praw.score)
    submissions_dict["num_comments"].append(submission_praw.num_comments)
    submissions_dict["created_utc"].append(submission_praw.created_utc)
    submissions_dict["selftext"].append(submission_praw.selftext)
    submissions_dict["is_original_content"].append(submission_praw.is_original_content)
    submissions_dict["distinguished"].append(submission_praw.distinguished)

## store each post's comment thread in single, separate csv file
submission_comments_csv_path = str(year) + '-' + subreddit + '-submission' + submission_id + '-comments.csv'
submission_comments_dict = {
    "comment_id" : [],
    "comment_parent_id" : [],
    "comment_body" : [],
    "comment_link_id" : [],
}

## For each comment in flattened thread, get data and store in dictionary, then in csv file
# extend comment tree entire way
submission_praw.comments.replace_more(limit=None)
# for each comment in flattened comment tree
for comment in submission_praw.comments.list():
    submission_comments_dict["comment_id"].append(comment.id)
    submission_comments_dict["comment_parent_id"].append(comment.parent_id)
    submission_comments_dict["comment_body"].append(comment.body)
    submission_comments_dict["comment_link_id"].append(comment.link_id)

# for each submission create separate csv comment file
pd.DataFrame(submission_comments_dict).to_csv(subredditdirpath + '\\' + submission_comments_csv_path,
            index=False)

## save all posts in csv file
# single csv with all submissions
pd.DataFrame(submissions_dict).to_csv(subredditdirpath + '\\' + submissions_csv_path,
                                              index=False)

action = f"\t\t[Info] Found submissions: {pd.DataFrame(submissions_dict).shape[0]}"
log_action(action)

action = f"\t\t[Info] Elapsed time: {time.time() - start_time: .2f}s"
log_action(action)

[Year]2022
	[Subreddit]conspiracy


KeyboardInterrupt: 